In [18]:
import requests
import re
import time
import os
import csv
import sys
import json
import pandas as pd

headers = {
    'Cookie': 'NewUniqueId=GUID_F6DCB57C_B736_54D3_F293_9DCFECB5536A; NewUniqueIdtmp1=GUID_07943413_1417_BDE5_B271_380E19E8C192; ' ,
    'Referer': 'https://m.weibo.cn/api/container/',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.122 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest'
}
data_Dict = {'data':[],'screen_name':[]}

url = 'https://m.weibo.cn/api/container/getIndex?containerid=100103type%3D1%26q%3D%23%E6%95%85%E5%AE%AB%E8%AE%B2%E5%9D%9B%23&page_type=searchall'
urlplus = 'https://m.weibo.cn/api/container/getIndex?containerid=100103type%3D1%26q%3D%23%E6%95%85%E5%AE%AB%E8%AE%B2%E5%9D%9B%23&page_type=searchall&page='

def Get_Screen_name(arg):
    data_Dict['screen_name'].append(arg)

def Get_CHN(text):
    dr = re.compile(r'<[^>]+>',re.S)
    dd = dr.sub('',text)
    data_Dict['data'].append(dd)

def get_longtext_page(id):
    extendurl='https://m.weibo.cn/statuses/extend?id={0}'.format(str(id))
    r = requests.get(extendurl, headers=headers)
    if r.status_code == 200:
        return r.json()
    
def parse_longtext_page(data):
    if data['ok']==1:
        Get_CHN(data['data']['longTextContent'])
    
def get_page(offset):
    params = {
        'page': offset,
    }
    try:
        if offset<=1:
            r = requests.get(url, headers=headers)
            if r.status_code == 200:
                return r.json()
        else:
            r = requests.get(urlplus+str(offset), headers=headers)
            if r.status_code == 200:
                return r.json()
    except requests.ConnectionError as e:
        print('error', e.args)

def parse_page(data):
    if data['ok']==1:
        for cardGroup in data['data']['cards']:
            if cardGroup['card_type']==9: 
                if cardGroup['mblog']['isLongText'] == True:
                    parse_longtext_page(get_longtext_page(cardGroup['mblog']['id']))  
                    Get_Screen_name(cardGroup['mblog']['user']['screen_name'])
                elif cardGroup['mblog']['isLongText'] == False:
                    Get_CHN(cardGroup['mblog']['text'])
                    Get_Screen_name(cardGroup['mblog']['user']['screen_name'])
    else:
        print('error')
        print(data)
        
def write_to_excel(content):
    pdData = pd.DataFrame(content)
    pdData.to_excel("#故宫讲坛#_20210414_100pages.xlsx",encoding='utf_8_sig')

def main():    
    for page in range(1,101):
        print("the current page is： {0}".format(page))
        jsondata = get_page(page)
        parse_page(jsondata)
        write_to_excel(data_Dict)
        time.sleep(5)

main()

the current page is： 1
the current page is： 2
the current page is： 3
the current page is： 4
the current page is： 5
the current page is： 6
the current page is： 7
the current page is： 8
the current page is： 9
the current page is： 10
the current page is： 11
the current page is： 12
the current page is： 13
the current page is： 14
the current page is： 15
the current page is： 16
the current page is： 17
the current page is： 18
the current page is： 19
the current page is： 20
the current page is： 21
the current page is： 22
the current page is： 23
the current page is： 24
the current page is： 25
the current page is： 26
the current page is： 27
error
{'ok': 0, 'msg': '这里还没有内容', 'data': {'cards': []}}


KeyboardInterrupt: 